<a href="https://colab.research.google.com/github/peterbabulik/QuantumMaterialsGenerator/blob/main/MaterialsDatasetGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tqdm cirq numpy pandas --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 70.4 MB/s eta 0:00:00


In [4]:
import sys
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import multiprocessing
import uuid
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# --- 2. Worker Function (Self-Contained) ---

def generate_single_sample(target_bg):
    """
    Worker function that contains the entire generator logic.
    This encapsulation ensures that imports and patches run INSIDE the worker process,
    avoiding conflicts with Dask/Joblib serialization.
    """
    # 1. CRITICAL FIX: Patch DaskFinder in the worker process
    import sys
    sys.meta_path = [x for x in sys.meta_path if "DaskFinder" not in str(type(x))]

    # 2. Imports (Local to worker)
    import cirq
    import numpy as np
    from sklearn.ensemble import RandomForestRegressor
    import sympy
    from scipy.optimize import minimize

    # 3. Class Definition (Local to worker)
    class QuantumMaterialsGenerator:
        def __init__(self, n_features=6, n_qubits=6):
            self.n_features = n_features
            self.n_qubits = n_qubits
            self.qubits = cirq.LineQubit.range(self.n_qubits)
            self.circuit = None
            self.params = []

            # Internal Database (V2 Ternary)
            self.data = [
                {'formula': 'Si',      'feat': [14, 0, 0,  1.0, 0.0, 0.0], 'bg': 1.12},
                {'formula': 'GaAs',    'feat': [31, 33, 0, 0.5, 0.5, 0.0], 'bg': 1.42},
                {'formula': 'GaN',     'feat': [31, 7, 0,  0.5, 0.5, 0.0], 'bg': 3.4},
                {'formula': 'InP',     'feat': [49, 15, 0, 0.5, 0.5, 0.0], 'bg': 1.34},
                {'formula': 'CdSe',    'feat': [48, 34, 0, 0.5, 0.5, 0.0], 'bg': 1.74},
                {'formula': 'ZnO',     'feat': [30, 8, 0,  0.5, 0.5, 0.0], 'bg': 3.37},
                {'formula': 'CuInSe2', 'feat': [29, 49, 34, 0.25, 0.25, 0.5], 'bg': 1.04},
                {'formula': 'CuGaSe2', 'feat': [29, 31, 34, 0.25, 0.25, 0.5], 'bg': 1.68},
                {'formula': 'AlGaAs',  'feat': [13, 31, 33, 0.1, 0.4, 0.5],   'bg': 1.80},
                {'formula': 'CdZnTe',  'feat': [48, 30, 52, 0.45, 0.05, 0.5], 'bg': 1.55},
                {'formula': 'MAPbI3',  'feat': [82, 53, 7,  0.2, 0.6, 0.2],   'bg': 1.55},
                {'formula': 'SiO2',    'feat': [14, 8, 0, 0.33, 0.67, 0.0],   'bg': 8.9},
                {'formula': 'Al2O3',   'feat': [13, 8, 0, 0.40, 0.60, 0.0],   'bg': 7.0},
                {'formula': 'TiO2',    'feat': [22, 8, 0, 0.33, 0.67, 0.0],   'bg': 3.2},
                {'formula': 'ZnO',     'feat': [30, 8, 0, 0.50, 0.50, 0.0],   'bg': 3.3},
            ]

            self._train_surrogate_model()

        def _train_surrogate_model(self):
            X = [d['feat'] for d in self.data]
            y = [d['bg'] for d in self.data]
            self.model = RandomForestRegressor(n_estimators=100, random_state=42)
            self.model.fit(X, y)

            self.periodic_table = {
                0: '', 1:'H', 5:'B', 6:'C', 7:'N', 8:'O', 13:'Al', 14:'Si',
                15:'P', 26:'Fe', 28:'Ni', 29:'Cu', 30:'Zn', 31:'Ga', 32:'Ge',
                33:'As', 34:'Se', 47:'Ag', 48:'Cd', 49:'In', 50:'Sn', 51:'Sb',
                52:'Te', 53:'I', 79:'Au', 82:'Pb', 83:'Bi'
            }

        def create_variational_circuit(self, layers=2):
            circuit = cirq.Circuit()
            circuit.append(cirq.H.on_each(self.qubits))
            for layer in range(layers):
                for i in range(self.n_qubits):
                    sx = sympy.Symbol(f'th_{layer}_{i}_x')
                    sy = sympy.Symbol(f'th_{layer}_{i}_y')
                    sz = sympy.Symbol(f'th_{layer}_{i}_z')
                    self.params.extend([sx, sy, sz])
                    circuit.append([cirq.rx(sx)(self.qubits[i]), cirq.ry(sy)(self.qubits[i]), cirq.rz(sz)(self.qubits[i])])
                for i in range(self.n_qubits):
                    circuit.append(cirq.CNOT(self.qubits[i], self.qubits[(i+1)%self.n_qubits]))
            self.circuit = circuit
            return circuit

        def quantum_to_features(self, param_values):
            resolver = cirq.ParamResolver({p: v for p, v in zip(self.params, param_values)})
            sim = cirq.Simulator()
            result = sim.simulate(self.circuit, resolver)
            probs = result.state_vector()

            val_a = np.sum(np.abs(probs[0:4]))
            atom_a = int(val_a * 83)
            val_b = np.sum(np.abs(probs[4:8]))
            atom_b = int(val_b * 83)
            val_c = np.sum(np.abs(probs[8:12]))
            atom_c = int(val_c * 83)

            total = np.abs(probs[16]) + np.abs(probs[17]) + np.abs(probs[18]) + 1e-9
            r1 = np.abs(probs[16]) / total
            r2 = np.abs(probs[17]) / total
            r3 = np.abs(probs[18]) / total

            return [atom_a, atom_b, atom_c, r1, r2, r3]

        def get_closest_real_element(self, atomic_num):
            available_atoms = list(self.periodic_table.keys())
            if atomic_num < 1: return 0, ""
            real_atoms = [a for a in available_atoms if a > 0]
            closest = min(real_atoms, key=lambda x: abs(x - atomic_num))
            return closest, self.periodic_table[closest]

        def _is_metal(self, atomic_number):
            metals = [13, 29, 30, 31, 48, 49, 50, 82, 83, 26, 28, 47, 79]
            return atomic_number in metals

        def snap_stoichiometry(self, ratio):
            standard_ratios = [0.2, 0.25, 0.33, 0.4, 0.5, 0.6, 0.66, 0.75, 0.8]
            closest = min(standard_ratios, key=lambda x: abs(x - ratio))
            if abs(ratio - closest) < 0.05: return closest
            return ratio

        def format_ternary_formula(self, atoms, ratios):
            element_counts = {}
            for atom, ratio in zip(atoms, ratios):
                if atom == 0: continue
                symbol = self.periodic_table.get(atom, "X")
                element_counts[symbol] = element_counts.get(symbol, 0) + ratio

            formula_parts = []
            for symbol, count in element_counts.items():
                if count > 0.01:
                    formula_parts.append(f"{symbol}{count:.2f}")
            return "".join(formula_parts)

        def calculate_stability_score(self, atoms, ratios):
            """
            Returns a score (0.0 to 1.0).
            1.0 means perfectly Charge Neutral (Stable).
            0.0 means highly unstable.
            """
            # Simplified oxidation states for common elements
            oxidation_states = {
                8: -2,  # Oxygen
                15: -3, # P (Phosphide)
                34: -2, # Se (Selenide)
                53: -1, # I (Iodide)
                # Metals (approximate common states)
                13: 3,  # Al
                26: 2,  # Fe (can be 3)
                28: 2,  # Ni
                29: 1,  # Cu (can be 2)
                30: 2,  # Zn
                31: 3,  # Ga
                47: 1,  # Ag
                48: 2,  # Cd
                49: 3,  # In
                32: 4   # Ge
            }

            net_charge = 0

            for atom, ratio in zip(atoms, ratios):
                if atom == 0: continue
                # Default to +2 if unknown metal, -2 if unknown non-metal
                state = oxidation_states.get(atom, 2 if atom < 50 else -2)
                net_charge += state * ratio

            # Physical Stability = How close is Net Charge to 0?
            deviation = abs(net_charge)
            score = np.exp(-deviation) # Map 0 -> 1.0, High -> 0.0
            return score

        def predict_properties(self, features):
            a_z, a_sym = self.get_closest_real_element(features[0])
            b_z, b_sym = self.get_closest_real_element(features[1])
            c_z, c_sym = self.get_closest_real_element(features[2])

            r1 = self.snap_stoichiometry(features[3])
            r2 = self.snap_stoichiometry(features[4])
            r3 = self.snap_stoichiometry(features[5])

            active_atoms = [z for z in [a_z, b_z, c_z] if z > 0]
            if active_atoms and all(self._is_metal(z) for z in active_atoms):
                atoms = [a_z, b_z, c_z]
                ratios = [r1, r2, r3]
                formula = self.format_ternary_formula(atoms, ratios)
                return {
                    'bandgap': 0.0,
                    'formula': f"{formula} (Metallic Alloy)",
                    'features': [a_z, b_z, c_z, r1, r2, r3],
                    'stability': 1.0
                }

            ai_input = [a_z, b_z, c_z, r1, r2, r3]
            predicted_bg = self.model.predict([ai_input])[0]

            atoms = [a_z, b_z, c_z]
            ratios = [r1, r2, r3]
            formula = self.format_ternary_formula(atoms, ratios)
            stability = self.calculate_stability_score(atoms, ratios)

            if not formula: formula = "Unknown"

            return {
                'bandgap': predicted_bg,
                'formula': formula,
                'features': ai_input,
                'stability': stability
            }

        def optimize_material(self, target_bandgap, steps=50):
            def objective(params):
                feats = self.quantum_to_features(params)
                props = self.predict_properties(feats)
                return (props['bandgap'] - target_bandgap) ** 2

            init_params = np.random.uniform(0, 2*np.pi, len(self.params))
            res = minimize(objective, init_params, method='COBYLA', options={'maxiter': steps}, tol=1e-3)
            final_feats = self.quantum_to_features(res.x)
            return self.predict_properties(final_feats)

    # --- Execution Logic ---
    gen = QuantumMaterialsGenerator()
    gen.create_variational_circuit(layers=2)

    # Fast optimization
    res = gen.optimize_material(target_bg, steps=15)

    features = res['features']
    elements = [int(features[0]), int(features[1]), int(features[2])]
    active_elements = [e for e in elements if e > 0]
    is_metal_flag = bool(active_elements and all(gen._is_metal(e) for e in active_elements))

    return {
        'uid': str(uuid.uuid4()),
        'formula': res['formula'],
        'target_bandgap': float(target_bg),
        'predicted_bandgap': float(res['bandgap']),
        'absolute_error': abs(res['bandgap'] - target_bg),
        'elements': str(active_elements),
        'is_metal': is_metal_flag
    }

def main():
    print("🚀 Starting Quantum Dataset Generation on Server...")
    print("ℹ️  Using Encapsulated Worker Logic to bypass Dask/Cirq conflicts.")

    # Configuration
    bandgaps = [round(x, 1) for x in np.arange(0.0, 4.1, 0.1)]
    SAMPLES_PER_BG = 10
    TOTAL_SAMPLES = len(bandgaps) * SAMPLES_PER_BG

    tasks = []
    for bg in bandgaps:
        tasks.extend([bg] * SAMPLES_PER_BG)

    print(f"🎯 Target: {TOTAL_SAMPLES} materials across {len(bandgaps)} bandgap steps.")
    print(f"⚡ Processing with {multiprocessing.cpu_count()} cores...")

    start_time = time.time()

    # Joblib Parallel Execution
    results = Parallel(n_jobs=-1, backend="loky")(
        delayed(generate_single_sample)(bg) for bg in tqdm(tasks, unit="mat")
    )

    end_time = time.time()
    duration = end_time - start_time

    print(f"✅ Generation Complete in {duration:.2f} seconds!")
    print(f"⚡ Rate: {TOTAL_SAMPLES / duration:.1f} materials/sec")

    # Save
    df = pd.DataFrame(results)
    cols = ['uid', 'formula', 'target_bandgap', 'predicted_bandgap', 'absolute_error', 'elements', 'is_metal']
    df = df[cols]

    filename = 'quantum_materials_dataset_v1.csv'
    df.to_csv(filename, index=False)
    print(f"💾 Saved to {filename}")
    print(df.head())

if __name__ == "__main__":
    main()

🚀 Starting Quantum Dataset Generation on Server...
ℹ️  Using Encapsulated Worker Logic to bypass Dask/Cirq conflicts.
🎯 Target: 410 materials across 41 bandgap steps.
⚡ Processing with 2 cores...


  0%|          | 0/410 [00:00<?, ?mat/s]

✅ Generation Complete in 353.73 seconds!
⚡ Rate: 1.2 materials/sec
💾 Saved to quantum_materials_dataset_v1.csv
                                    uid                              formula  \
0  478bfd7c-2ef0-4263-a64b-0aa6bed316dd        Fe0.60Al0.40 (Metallic Alloy)   
1  89c549ac-8639-4cac-bfe2-5400a626e291        Fe0.90Cd0.08 (Metallic Alloy)   
2  11bce1fb-7c18-4873-924f-4416cdfaf4b1        Fe0.80Ag0.20 (Metallic Alloy)   
3  1a88951e-8c37-42fb-8e90-727c00979de0  Ga0.11Fe0.66Sn0.25 (Metallic Alloy)   
4  1fb5670f-455a-44fa-982b-80ca591438ef  Au0.33Fe0.40Pb0.25 (Metallic Alloy)   

   target_bandgap  predicted_bandgap  absolute_error      elements  is_metal  
0             0.0                0.0             0.0  [26, 13, 26]      True  
1             0.0                0.0             0.0  [26, 48, 26]      True  
2             0.0                0.0             0.0  [26, 26, 47]      True  
3             0.0                0.0             0.0  [31, 26, 50]      True  
4            

In [5]:
from IPython.display import FileLink
FileLink('quantum_materials_dataset_v1.csv')

/content/quantum_materials_dataset_v1.csv